In [1]:
import json
import pandas as pd

from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [2]:
#location = "boston"
#location = "geneva"
location = "hong-kong"
index_name = "airbnb_history_" + location

In [3]:
from elasticsearch.helpers import scan

res = scan(
    es,
    index=index_name,
    query={"query": { "match_all" : {}}}
)

In [4]:
data = list(res)

In [5]:
len(data)

304041

In [6]:
# appending rows 
docs = []
for item in data:  
    doc = {
        'id': item['_id'],
        'index': item['_index'],
        'listing_url': item['_source']['listing_url'],
        'scrape_id': item['_source']['scrape_id'],
        'last_scraped': item['_source']['last_scraped'],
        'crawled_date': item['_source']['crawled_date'],
        'name': item['_source']['name'],
        'host_id': item['_source']['host_id'],
        'host_is_superhost': item['_source']['host_is_superhost'],
        'host_identity_verified': item['_source']['host_identity_verified'],
        'room_type': item['_source']['room_type'],
        'accommodates': item['_source']['accommodates'],
        'guests_included': item['_source']['guests_included'],
        'minimum_nights': item['_source']['minimum_nights'],
        'maximum_nights': item['_source']['maximum_nights'],
        'calendar_updated': item['_source']['calendar_updated'],
        'instant_bookable': item['_source']['instant_bookable'],
        'is_business_travel_ready': item['_source']['is_business_travel_ready'],
        'cancellation_policy': item['_source']['cancellation_policy'],
        'price': item['_source']['price'],
        'availability_30': item['_source']['availability_30'],
        'availability_60': item['_source']['availability_60'],
        'availability_90': item['_source']['availability_90'],
        'availability_365': item['_source']['availability_365'],
        'first_review': item['_source']['first_review'],
        'last_review': item['_source']['last_review'],
        'review_scores_rating': item['_source']['review_scores_rating'],
        'review_scores_accuracy': item['_source']['review_scores_accuracy'],
        'review_scores_cleanliness': item['_source']['review_scores_cleanliness'],
        'review_scores_checkin': item['_source']['review_scores_checkin'],
        'review_scores_communication': item['_source']['review_scores_communication'],
        'review_scores_location': item['_source']['review_scores_location'],
        'review_scores_value': item['_source']['review_scores_value'],
        'overall_rating': item['_source']['overall_rating']
    }
    
    docs.append(doc) 

In [7]:
len(docs)

304041

In [8]:
def divide_chunks(docs, n): 
    """ Split list of documents into n-sized chunks
    :param l: list
    :param n: num of paritions
    """
    org_size = len(docs)
    chunk_size = org_size / float(n)
    print(org_size)
    print(chunk_size)
    
    return [ docs[int(round(chunk_size * i)): int(round(chunk_size * (i + 1)))] for i in range(n) ]

In [9]:
partitions = divide_chunks(docs, 3)

304041
101347.0


In [10]:
len(partitions[0])

101347

In [11]:
partitions[0][:2]

[{'id': 'IiF-yXcBLDmwrO9FxvMT',
  'index': 'airbnb_history_hong-kong',
  'listing_url': 'https://www.airbnb.com/rooms/12814144',
  'scrape_id': 20160806232118,
  'last_scraped': '20160807',
  'crawled_date': '20160806',
  'name': '温馨的住家',
  'host_id': 69782768,
  'host_is_superhost': 'f',
  'host_identity_verified': 't',
  'room_type': 'Entire home/apt',
  'accommodates': 6,
  'guests_included': 1,
  'minimum_nights': 1,
  'maximum_nights': 1125,
  'calendar_updated': '3 months ago',
  'instant_bookable': 't',
  'is_business_travel_ready': 'f',
  'cancellation_policy': 'flexible',
  'price': 719.0,
  'availability_30': 30,
  'availability_60': 60,
  'availability_90': 90,
  'availability_365': 365,
  'first_review': '19910101',
  'last_review': '0',
  'review_scores_rating': 0.0,
  'review_scores_accuracy': 0.0,
  'review_scores_cleanliness': 0.0,
  'review_scores_checkin': 0.0,
  'review_scores_communication': 0.0,
  'review_scores_location': 0.0,
  'review_scores_value': 0.0,
  'over

In [12]:
len(partitions[1])

101347

In [13]:
len(partitions[2])

101347

In [14]:
import json
i = 0

for p in partitions:
    
    with open('airbnb_' + location + "_" + str(i) + '.json', 'w') as f:
        json.dump(p, f)
        
    i = i+1